In [10]:
import pandas as pd
import joblib

In [30]:
from src.data_fetcher import get_fpl_data
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
data = get_fpl_data()
data


(     can_transact  can_select  chance_of_playing_next_round  \
 0            True       False                           0.0   
 1            True       False                           0.0   
 2            True       False                           0.0   
 3            True       False                         100.0   
 4            True       False                           0.0   
 ..            ...         ...                           ...   
 799          True       False                         100.0   
 800          True       False                           NaN   
 801          True       False                           NaN   
 802          True       False                         100.0   
 803          True       False                           NaN   
 
      chance_of_playing_this_round       code  cost_change_event  \
 0                             0.0     438098                  0   
 1                             0.0     205651                  0   
 2                        

In [38]:
import requests

fpl_api_url = "https://fantasy.premierleague.com/api"

def get_all_players():
    url = f"{fpl_api_url}/bootstrap-static/"
    try:
        response = requests.get(url)
        response.raise_for_status()  # raises HTTPError if not 200
        return response.json()["elements"]
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to fetch players: {e}")
        return []

In [45]:
data=get_all_players()
data

[{'can_transact': True,
  'can_select': False,
  'chance_of_playing_next_round': 0,
  'chance_of_playing_this_round': 0,
  'code': 438098,
  'cost_change_event': 0,
  'cost_change_event_fall': 0,
  'cost_change_start': -1,
  'cost_change_start_fall': 1,
  'dreamteam_count': 0,
  'element_type': 3,
  'ep_next': '0.0',
  'ep_this': '0.0',
  'event_points': 0,
  'first_name': 'Fábio',
  'form': '0.0',
  'id': 1,
  'in_dreamteam': False,
  'news': 'Has joined Portuguese side FC Porto on loan for the 2024/25 season',
  'news_added': '2024-08-29T11:06:25.241953Z',
  'now_cost': 54,
  'photo': '438098.jpg',
  'points_per_game': '0.0',
  'removed': False,
  'second_name': 'Ferreira Vieira',
  'selected_by_percent': '0.0',
  'special': False,
  'squad_number': None,
  'status': 'u',
  'team': 1,
  'team_code': 3,
  'total_points': 0,
  'transfers_in': 439,
  'transfers_in_event': 0,
  'transfers_out': 2825,
  'transfers_out_event': 0,
  'value_form': '0.0',
  'value_season': '0.0',
  'web_name'

In [13]:
import os
print(os.path.exists("/Users/joeriksson/fehrman-git/fpl_bot/models/fpl_ai_model.pkl"))  # True if correct

True


In [14]:
model = joblib.load("/Users/joeriksson/fehrman-git/fpl_bot/models/fpl_ai_model.pkl")

/Users/joeriksson/fehrman-git/fpl_bot/.venv/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.1.3 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
# 3. Simulate a few rows of sample FPL API data
players = pd.DataFrame({
    "first_name": ["Erling", "Bukayo", "Alisson"],
    "second_name": ["Haaland", "Saka", "Becker"],
    "now_cost": [120, 85, 55],  # in tenths of millions
    "minutes": [3000, 2900, 3200],
    "form": [9.2, 8.1, 5.5],
    "transfers_in": [400000, 350000, 200000],
    "goals_scored": [30, 14, 0],
    "assists": [5, 11, 0],
    "yellow_cards": [3, 2, 1],
    "threat": [80.5, 65.0, 10.0],
    "influence": [90.0, 72.5, 50.0]
})

# 4. Define features
FEATURE_COLUMNS = [
    "now_cost", "minutes", "form", "transfers_in",
    "goals_scored", "assists", "yellow_cards", "threat", "influence"
]

# 5. Predict
X = players[FEATURE_COLUMNS]
players["expected_points"] = model.predict(X)

# 6. Show results
players[["first_name", "second_name", "expected_points"]]

,first_name,second_name,expected_points
0,Erling,Haaland,89.425396
1,Bukayo,Saka,111.263998
2,Alisson,Becker,145.517962


In [16]:
players

,first_name,second_name,now_cost,minutes,form,transfers_in,goals_scored,assists,yellow_cards,threat,influence,expected_points
0,Erling,Haaland,120,3000,9.2,400000,30,5,3,80.5,90.0,89.425396
1,Bukayo,Saka,85,2900,8.1,350000,14,11,2,65.0,72.5,111.263998
2,Alisson,Becker,55,3200,5.5,200000,0,0,1,10.0,50.0,145.517962


In [23]:
import requests
import pandas as pd

fpl_api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
def get_player_history(player_id):
    url = f"{fpl_api_url}/elements/{player_id}/"
    response = requests.get(url)
    if response.status_code != 200:
        return None
    return pd.DataFrame(response.json()["history"])

In [62]:
import requests
import pandas as pd
import time
import os


BASE_API = "https://fantasy.premierleague.com/api"

def get_fpl_data():
    url = f"{BASE_API}/bootstrap-static/"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        players = pd.DataFrame(data['elements'])
        teams = pd.DataFrame(data['teams'])
        return players, teams
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

def get_player_history(player_id):
    url = f"{BASE_API}/element-summary/{player_id}/"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if "history" not in data:
            raise ValueError("Missing 'history' key in response")

        return pd.DataFrame(data["history"])

    except Exception as e:
        print(f"❌ Failed to fetch history for player {player_id}: {e}")
        return None

def get_all_players():
    try:
        response = requests.get(f"{BASE_API}/bootstrap-static/")
        response.raise_for_status()
        bootstrap = response.json()
        return pd.DataFrame(bootstrap["elements"])
    except Exception as e:
        print(f"❌ Failed to get all players: {e}")
        return pd.DataFrame()

def fetch_weekly_data(save_path="data/processed/player_gameweeks.csv", limit=10):
    """
    Download weekly-level player performance data for active players.
    Args:
        save_path (str): Path to save output CSV
        limit (int): Number of active players to fetch (default: 10)
    Returns:
        pd.DataFrame: Merged gameweek history
    """
    all_players = get_all_players()
    if all_players.empty:
        raise ValueError("No players found from API.")

    # Filter only 'active' players (status == 'a')
    active_players = all_players[all_players["status"] == "a"].copy()
    if active_players.empty:
        raise ValueError("No active players found.")

    print(f"🔍 Found {len(active_players)} active players. Limiting to {limit} for this run...")

    all_data = []

    for i, (_, player) in enumerate(active_players.iterrows()):
        if i >= limit:
            break

        pid = player["id"]
        name = f"{player['first_name']} {player['second_name']}"
        print(f"⬇️ Fetching history for {name} (id={pid})")

        history = get_player_history(pid)
        if history is not None and not history.empty:
            history["player_id"] = pid
            history["name"] = name
            all_data.append(history)

        time.sleep(0.8)  # be kind to FPL API

    if not all_data:
        raise ValueError("No player history was fetched.")

    df = pd.concat(all_data, ignore_index=True)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    df.to_csv(save_path, index=False)
    print(f"✅ Data saved to {save_path}")
    return df

In [63]:
df = fetch_weekly_data(limit=10)
print(df.head())

🔍 Found 561 active players. Limiting to 10 for this run...
⬇️ Fetching history for Kai Havertz (id=4)
⬇️ Fetching history for Jorge Luiz Frello Filho (id=7)
⬇️ Fetching history for Jakub Kiwior (id=8)
⬇️ Fetching history for Gabriel Martinelli Silva (id=9)
⬇️ Fetching history for Ethan Nwaneri (id=12)
⬇️ Fetching history for Martin Ødegaard (id=13)
⬇️ Fetching history for David Raya Martin (id=15)
⬇️ Fetching history for Declan Rice (id=16)
⬇️ Fetching history for Bukayo Saka (id=17)
⬇️ Fetching history for Thomas Partey (id=20)


FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [64]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Models to compare
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Load data
df = pd.read_csv("data/processed/player_gameweeks.csv")

# Define features and target
features = ['minutes', 'goals_scored', 'assists', 'bonus', 'ict_index']
X = df[features]
y = df["total_points"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    results.append({
        "Model": name,
        "MAE": round(mae, 3),
        "MSE": round(mse, 3),
        "R²": round(r2, 3)
    })

    # Save best model
    if name == "Random Forest":
        joblib.dump((model, features), "models/week_model.pkl")

# Show results
results_df = pd.DataFrame(results)
print(results_df)


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/player_gameweeks.csv'